In [19]:
import numpy as np
from typing import Tuple
from scipy.stats import mode

from sklearn.metrics import confusion_matrix
import seaborn as sns

csv_path = 'iris.csv'

In [18]:
#pip install seaborn

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 293 kB 1.4 MB/s eta 0:00:01
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [6]:
def load_csv(csv_path: str) -> Tuple[np.ndarray, np.ndarray]:
    np.random.seed(42)
    dataset = np.genfromtxt(csv_path, delimiter=',')
    np.random.shuffle(dataset)
    x,y = dataset[:,:-1], dataset[:, -1]
    return x,y

In [7]:
x,y = load_csv(csv_path)

x.shape, y.shape

((150, 4), (150,))

In [6]:
np.mean(x, axis=0),np.var(x, axis=0)

(array([nan, nan, nan, nan]), array([nan, nan, nan, nan]))

In [7]:
np.nanmean(x, axis=0),np.nanvar(x, axis=0)

(array([ 355.46503497, -280.09189189,    2.95      ,   21.74726027]),
 array([1.73561968e+07, 1.18405444e+07, 1.51049922e+04, 6.11729208e+04]))

In [8]:
x[np.isnan(x)] = 3.5

In [9]:
y = np.delete(y, np.where(x < 0.0)[0], axis=0)
y = np.delete(y, np.where(x > 10.0)[0], axis=0)
x = np.delete(x, np.where(x < 0.0)[0], axis=0)
x = np.delete(x, np.where(x > 10.0)[0], axis=0)

x.shape,y.shape

((144, 4), (144,))

In [10]:
def train_test_split(features: np.ndarray,
                     labels: np.ndarray,
                     test_plit_ratio: float) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    test_size = int(len(features) * test_plit_ratio)
    train_size = len(features) - test_size
    assert len(features) == test_size + train_size, "Size mismatch!"

    x_train,y_train = features[:train_size,:],labels[:train_size]
    x_test,y_test = features[train_size:train_size+test_size,:], labels[train_size:train_size + test_size]

    return x_train,y_train,x_test,y_test

In [11]:
x_train,y_train,x_test,y_test = train_test_split(x, y, 0.2)

In [13]:
def euclidean(points:np.ndarray, element_of_x:np.ndarray) -> np.ndarray:
    return np.sqrt(np.sum((points - element_of_x)**2, axis=1))

In [14]:
def predict(x_train: np.ndarray,
            y_train:np.ndarray,
            x_test:np.ndarray,
            k:int) -> np.ndarray:
    labels_pred = [];
    for x_test_element in x_test:
        distances = euclidean(x_train, x_test_element)
        distances = np.array(sorted(zip(distances, y_train)))

        label_pred = mode(distances[:k,1], keepdims=False).mode
        labels_pred.append(label_pred)

    return np.array(labels_pred,dtype=np.int64)

In [22]:
y_preds = predict(x_train, y_train, x_test, 3)


In [23]:
def accuracy(y_test:np.ndarray, y_preds:np.ndarray) -> float:
    true_positive = (y_test == y_preds).sum()
    return true_positive/len(y_test) * 100

accuracy(y_test, y_preds)

89.28571428571429

In [ ]:
def plot_confusion_matrix(y_test:np.ndarray, y_preds:np.ndarray):
    conf_matrix = confusion_matrix(y_test,y_preds)
    sns.heatmap(conf_matrix, annot=True)

